#### Block 1: Import Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Block 2: Import Functions

In [ ]:
from functions.fn_set_env import Param
from functions.fn_set_value import *
from functions.fn_algorithm import *
from functions.fn_consumption import *
from functions.fn_metrics import *
from functions.fn_simulation import *

#### Block 3: Reference

In [ ]:
def run_simulation_ref(params, user_item_utility, reserve_utilities, algo_1, algo_2):
    """
    Reference Method: 
    algo_1 and algo_2 for this method are identical recommendation algorithms.
    Both algorithms learn from the entirety of the user data, ensuring no differences in how they learn or perform.
    This serves as the control group for analyzing potential biases, as no bias is introduced by the learning process.

    Args:
        params: Contains the enviroment set up.
        user_item_utility (function/array): A utility function or matrix providing user-utility values for items.
        reserve_utilities (array): Reserve_utilities for users.
        algo_1 (callable): The first recommendation algorithm; signature should match how it's called below.
        algo_2 (callable): The second recommendation algorithm; signature should match how it's called below.

    Returns:
        avg_c_algo_1 (np.array): [mean, 2.5% quantile, 97.5% quantile, variance] of take-up rates for algo_1.
        avg_c_algo_2 (np.array): Same structure as avg_c_algo_1 but for algo_2.
        avg_TE (np.array): Treatment effect (algo_1 - algo_2) summary stats.
        avg_pct_TE (np.array): Percentage-based treatment effect ((algo_1 - algo_2)/algo_1) summary stats.
    """

    # Initialize results lists
    avg_c_algo_1_list = []
    avg_c_algo_2_list = []

    # Pre-generate user assignment matrix for all simulation rounds (num of rounds = params.B)
    # Each row in user_assignments_matrix is a array of length num_users indicating which users are assigned to algo_2 (1) or algo_1 (0).
    user_assignments_matrix = np.random.randint(0, 2, (params.B, params.num_users)).astype(bool)

    # Number of new items in each period
    n_new = params.num_items_per_period

    # For each round of simulation
    for b in range(params.B):
        # Get user assignments for this simulation round
        user_assignments = user_assignments_matrix[b]
        
        # Generate random noise for each period and user
        # Shape: (num_periods x num_users x (num_periods * num_items_per_period))
        noise = np.random.normal(0, 1,(params.num_periods, params.num_users, params.num_periods * params.num_items_per_period))
        
        # Initialize the user-item interaction matrix: 0 indicates no consumption, 1 indicates consumption
        # It’s essentially the interaction history that the algorithms use to learn user preferences.
        # Shape: (num_users x num_items)
        interaction_matrix = np.zeros((params.num_users, params.num_items))

        # Record previous consumption to keep track of all items consumed by each user after the initial period
        # Used later to calculate take up rate
        prev_consumed_items = [[] for _ in range(params.num_users)]

        for t in range(params.num_periods):

            #### Introduce new goods
            # list(range(t * n_new, (t + 1) * n_new)) generates a list of new item indices introduced at period t. 
            # For example, if n_new = 5 and t = 2, this would produce [10, 11, 12, 13, 14].
            # np.repeat(..., params.num_users, axis=0) replicates this list so that each user has the same set of new items to choose from initially.
            new_items = np.repeat([list(range(t * n_new, (t + 1) * n_new))], params.num_users, axis=0)
            
            # np.apply_along_axis(np.random.shuffle, 1, new_items) shuffles the order of these new item indices independently for each user.
            np.apply_along_axis(np.random.shuffle, 1, new_items)
            
            # Initialize recommended items list for each user
            recommended_items = [[] for _ in range(params.num_users)]

            #### Recommendation step (happens only after initial periods)
            # Update the training data every training_frequency periods:
            if t % params.training_frequency == 0 and t >= params.initial_periods:
                # Use item interation data for each user up to the current period (t * n_new) as training data
                # :(t * n_new) reflects all the items introduced up to the start of period t
                training_data = interaction_matrix[:, :(t * n_new)]
                
                # Recommended_items: Matrix of ranked item IDs recommended to each user.
                recommended_items_1 = algo_1(training_data, noise[t,:, :(t * n_new)])
                recommended_items_2 = algo_2(training_data, noise[t,:, :(t * n_new)])

                # Merge the two algorithms' recommendation list
                recommended_items = recommended_items_1.copy()
                recommended_items[user_assignments] = recommended_items_2[user_assignments]

            #### Consumption step
            # Simulate user consumption
            # chosen_items reflects ID of the item each user chooses to consume, 
            # where -1 indicates that user does not consume any item
            if t <  params.initial_periods:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)
            else:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)

            # Update the user-item interaction in interaction_matrix and prev_consumed_items
            for user_id, chosen_item in enumerate(chosen_items):
                prev_consumed_items[user_id].append(chosen_item)
                if chosen_item != -1:
                    interaction_matrix[user_id,chosen_item] = 1
                    
        # Calculate the average take-up rate
        [avg_c_algo_1,avg_c_algo_2] = avg_take_up_rate_by_period(prev_consumed_items, user_assignments, params)
        avg_c_algo_1_list.append(np.mean(avg_c_algo_1[params.initial_periods:]))
        avg_c_algo_2_list.append(np.mean(avg_c_algo_2[params.initial_periods:]))
        
        if b < params.B:
            # Print the count of the simulation with replacement
            print(f"Simulation {b + 1} of {params.B} completed", end='\r', flush=True)
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
        else:
            print(f"Simulation {b + 1} of {params.B} completed")
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")

    # Calculate the average take up rate for algo_1 and algo_2 across simulations
    avg_c_algo_1 = np.zeros(4)
    avg_c_algo_1[0] = np.mean(avg_c_algo_1_list, axis=0)
    avg_c_algo_1[1] = np.quantile(avg_c_algo_1_list,0.025)
    avg_c_algo_1[2] = np.quantile(avg_c_algo_1_list,0.975)
    avg_c_algo_1[3] = np.var(avg_c_algo_1, axis=0)

    avg_c_algo_2 = np.zeros(4)
    avg_c_algo_2[0] = np.mean(avg_c_algo_2_list, axis=0)
    avg_c_algo_2[1] = np.quantile(avg_c_algo_2_list,0.025)
    avg_c_algo_2[2] = np.quantile(avg_c_algo_2_list,0.975)
    avg_c_algo_2[3] = np.var(avg_c_algo_2, axis=0)
    
    # Calculate the average Treatment effect (algo_1 - algo_2) across simulations
    avg_TE_list = np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list)
    avg_TE = np.zeros(4)
    avg_TE[0] = np.mean(avg_TE_list, axis=0)
    avg_TE[1] = np.quantile(avg_TE_list,0.025)
    avg_TE[2] = np.quantile(avg_TE_list,0.975)
    avg_TE[3] = np.var(avg_TE_list, axis=0)
    
    # Calculate the average percentage Treatment effect across simulations
    avg_pct_TE_list = (np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list))/np.array(avg_c_algo_1_list)
    avg_pct_TE = np.zeros(4)
    avg_pct_TE[0] = np.mean(avg_pct_TE_list, axis=0)
    avg_pct_TE[1] = np.quantile(avg_pct_TE_list,0.025)
    avg_pct_TE[2] = np.quantile(avg_pct_TE_list,0.975)
    avg_pct_TE[3] = np.var(avg_pct_TE_list, axis=0)
    
    return avg_c_algo_1, avg_c_algo_2, avg_TE, avg_pct_TE

#### Block 4: Naive experiment

In [ ]:
def run_simulation_naive(params, user_item_utility, reserve_utilities, algo_1, algo_2, treatment_percentage):
    """
    Naive Method: 
    Both algo_1 and algo_2 learn from the same user data.
    All users interact with the same pool of items.
    
    Args:
        params: Contains the enviroment set up.
        user_item_utility (function/array): A utility function or matrix providing user-utility values for items.
        reserve_utilities (array): Reserve_utilities for users.
        algo_1 (callable): The first recommendation algorithm; signature should match how it's called below.
        algo_2 (callable): The second recommendation algorithm; signature should match how it's called below.
        treatment_percentage (float): Percentage of treated user

    Returns:
        avg_c_algo_1 (np.array): [mean, 2.5% quantile, 97.5% quantile, variance] of take-up rates for algo_1.
        avg_c_algo_2 (np.array): Same structure as avg_c_algo_1 but for algo_2.
        avg_TE (np.array): Treatment effect (algo_1 - algo_2) summary stats.
        avg_pct_TE (np.array): Percentage-based treatment effect ((algo_1 - algo_2)/algo_1) summary stats.
    """

    # Initialize results lists
    avg_c_algo_1_list = []
    avg_c_algo_2_list = []

    # Pre-generate user assignment matrix for all simulation rounds (num of rounds = params.B)
    # Each row in user_assignments_matrix is a array of length num_users indicating which users are assigned to algo_2 (1) or algo_1 (0).
    user_assignments_matrix = np.random.choice(
            [0, 1], size=(params.B, params.num_users), p=[1-treatment_percentage, treatment_percentage]
        ).astype(bool)    
    

    # Number of new items in each period
    n_new = params.num_items_per_period

    # For each round of simulation
    for b in range(params.B):
        # Get user assignments for this simulation round
        user_assignments = user_assignments_matrix[b]

    
        # Generate random noise for each period and user
        # Shape: (num_periods x num_users x (num_periods * num_items_per_period))
        noise = np.random.normal(0, 1,(params.num_periods,params.num_users, params.num_periods * params.num_items_per_period))
        
        # Initialize the user-item interaction matrix: 0 indicates no consumption, 1 indicates consumption
        # It’s essentially the interaction history that the algorithms use to learn user preferences.
        # Shape: (num_users x num_items)
        interaction_matrix = np.zeros((params.num_users, params.num_items))

        # Record previous consumption to keep track of all items consumed by each user after the initial period
        # Used later to calculate take up rate
        prev_consumed_items = [[] for _ in range(params.num_users)]

        for t in range(params.num_periods):

            #### Introduce new goods
            # list(range(t * n_new, (t + 1) * n_new)) generates a list of new item indices introduced at period t. 
            # For example, if n_new = 5 and t = 2, this would produce [10, 11, 12, 13, 14].
            # np.repeat(..., params.num_users, axis=0) replicates this list so that each user has the same set of new items to choose from initially.
            new_items = np.repeat([list(range(t * n_new, (t + 1) * n_new))], params.num_users, axis=0)
            
            # np.apply_along_axis(np.random.shuffle, 1, new_items) shuffles the order of these new item indices independently for each user.
            np.apply_along_axis(np.random.shuffle, 1, new_items)
            
            # Initialize recommended items list for each user
            recommended_items = [[] for _ in range(params.num_users)]

            #### Recommendation step (happens only after initial periods)
            # Update the training data every training_frequency periods:
            if t % params.training_frequency == 0 and t >= params.initial_periods:
                # Use item interation data for each user up to the current period (t * n_new) as training data
                # :(t * n_new) reflects all the items introduced up to the start of period t
                training_data = interaction_matrix[:, :(t * n_new)]
                
                # Recommended_items: Matrix of ranked item IDs recommended to each user.
                recommended_items_1 = algo_1(training_data, noise[t,:, :(t * n_new)])
                recommended_items_2 = algo_2(training_data, noise[t,:, :(t * n_new)])

                # Merge the two algorithms' recommendation list
                recommended_items = recommended_items_1.copy()
                recommended_items[user_assignments] = recommended_items_2[user_assignments]

            #### Consumption step
            # Simulate user consumption
            # chosen_items reflects ID of the item each user chooses to consume, 
            # where -1 indicates that user does not consume any item
            if t <  params.initial_periods:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)
            else:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)

            # Update the user-item interaction in interaction_matrix and prev_consumed_items
            for user_id, chosen_item in enumerate(chosen_items):
                prev_consumed_items[user_id].append(chosen_item)
                if chosen_item != -1:
                    interaction_matrix[user_id,chosen_item] = 1
                    
        # Calculate the average take-up rate
        [avg_c_algo_1,avg_c_algo_2] = avg_take_up_rate_by_period(prev_consumed_items, user_assignments, params)
        avg_c_algo_1_list.append(np.mean(avg_c_algo_1[params.initial_periods:]))
        avg_c_algo_2_list.append(np.mean(avg_c_algo_2[params.initial_periods:]))
        
        if b < params.B:
            # Print the count of the simulation with replacement
            print(f"Simulation {b + 1} of {params.B} completed", end='\r', flush=True)
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
        else:
            print(f"Simulation {b + 1} of {params.B} completed")
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")

    # Calculate the average take up rate for algo_1 and algo_2 across simulations
    avg_algo_1 = np.zeros(4)
    avg_algo_1[0] = np.mean(avg_c_algo_1_list, axis=0)
    avg_algo_1[1] = np.quantile(avg_c_algo_1_list,0.025)
    avg_algo_1[2] = np.quantile(avg_c_algo_1_list,0.975)
    avg_algo_1[3] = np.var(avg_algo_1, axis=0)
    
    avg_algo_2 = np.zeros(4)
    avg_algo_2[0] = np.mean(avg_c_algo_2_list, axis=0)
    avg_algo_2[1] = np.quantile(avg_c_algo_2_list,0.025)
    avg_algo_2[2] = np.quantile(avg_c_algo_2_list,0.975)
    avg_algo_2[3] = np.var(avg_algo_2, axis=0)
    
    # Calculate the average Treatment effect (algo_1 - algo_2) across simulations
    avg_TE_list = np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list)
    avg_TE = np.zeros(4)
    avg_TE[0] = np.mean(avg_TE_list, axis=0)
    avg_TE[1] = np.quantile(avg_TE_list,0.025)
    avg_TE[2] = np.quantile(avg_TE_list,0.975)
    avg_TE[3] = np.var(avg_TE_list, axis=0)
    
    # Calculate the average percentage Treatment effect across simulations
    avg_pct_TE_list = (np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list))/np.array(avg_c_algo_1_list)
    avg_pct_TE = np.zeros(4)
    avg_pct_TE[0] = np.mean(avg_pct_TE_list, axis=0)
    avg_pct_TE[1] = np.quantile(avg_pct_TE_list,0.025)
    avg_pct_TE[2] = np.quantile(avg_pct_TE_list,0.975)
    avg_pct_TE[3] = np.var(avg_pct_TE_list, axis=0)
    
    return treatment_percentage, avg_algo_1, avg_algo_2, avg_TE, avg_pct_TE

#### Block 5: Data-diverted experiment

In [ ]:
def run_simulation_data_diverted(params, user_item_utility, reserve_utilities, algo_1, algo_2, treatment_percentage):
    """
    Data-diverted Method: 
    Algo_1 and algo_2 each keeps a separate interaction history for their own users, and learns from their own interaction history. 
    Users interact with the same pool of items.
    
    Args:
        params: Contains the enviroment set up.
        user_item_utility (function/array): A utility function or matrix providing user-utility values for items.
        reserve_utilities (array): Reserve_utilities for users.
        algo_1 (callable): The first recommendation algorithm; signature should match how it's called below.
        algo_2 (callable): The second recommendation algorithm; signature should match how it's called below.
        treatment_percentage (float): Percentage of treated user

    Returns:
        avg_c_algo_1 (np.array): [mean, 2.5% quantile, 97.5% quantile, variance] of take-up rates for algo_1.
        avg_c_algo_2 (np.array): Same structure as avg_c_algo_1 but for algo_2.
        avg_TE (np.array): Treatment effect (algo_1 - algo_2) summary stats.
        avg_pct_TE (np.array): Percentage-based treatment effect ((algo_1 - algo_2)/algo_1) summary stats.
    """

    # Initialize results lists
    avg_c_algo_1_list = []
    avg_c_algo_2_list = []

    # Pre-generate user assignment matrix for all simulation rounds (num of rounds = params.B)
    # Each row in user_assignments_matrix is a array of length num_users indicating which users are assigned to algo_2 (1) or algo_1 (0).
    user_assignments_matrix = np.random.choice(
        [0, 1], size=(params.B, params.num_users), p=[1-treatment_percentage, treatment_percentage]
    ).astype(bool)

    

    # Number of new items in each period
    n_new = params.num_items_per_period

    # For each round of simulation
    for b in range(params.B):
        # Get user assignments for this simulation round
        user_assignments = user_assignments_matrix[b]
        
        # Generate random noise for each period and user
        # Shape: (num_periods x num_users x (num_periods * num_items_per_period))
        noise = np.random.normal(0, 1,(params.num_periods,params.num_users, params.num_periods * params.num_items_per_period))

        # Generate user-item interaction matrix for each algorithm : 0 indicates no consumption, 1 indicates consumption
        # It’s essentially the interaction history that the algorithms use to learn user preferences.
        # Shape: (num_users x num_items)
        interaction_matrix_algo_1 = np.zeros((params.num_users, params.num_items))
        interaction_matrix_algo_2 = np.zeros((params.num_users, params.num_items))

        # Record previous consumption to keep track of all items consumed by each user after the initial period
        # Used later to calculate take up rate
        prev_consumed_items = [[] for _ in range(params.num_users)]

        for t in range(params.num_periods):


            #### Introduce new goods
            # list(range(t * n_new, (t + 1) * n_new)) generates a list of new item indices introduced at period t. 
            # For example, if n_new = 5 and t = 2, this would produce [10, 11, 12, 13, 14].
            # np.repeat(..., params.num_users, axis=0) replicates this list so that each user has the same set of new items to choose from initially.
            new_items = np.repeat([list(range(t * n_new, (t + 1) * n_new))], params.num_users, axis=0)
            
            # np.apply_along_axis(np.random.shuffle, 1, new_items) shuffles the order of these new item indices independently for each user.
            np.apply_along_axis(np.random.shuffle, 1, new_items)
            
            # Initialize recommended items list for each user
            recommended_items = [[] for _ in range(params.num_users)]

            #### Recommendation step (happens only after initial periods)
            # Update the training data every training_frequency periods:
            if t % params.training_frequency == 0 and t >= params.initial_periods:
                # Use item interation data for each user up to the current period (t * n_new) as training data
                # :(t * n_new) reflects all the items introduced up to the start of period t
                training_data_1 = interaction_matrix_algo_1[:, :(t * n_new)].copy()
                training_data_2 = interaction_matrix_algo_2[:, :(t * n_new)].copy()

                # Recommended_items: Matrix of ranked item IDs recommended to each user.
                recommended_items_1 = algo_1(training_data_1, noise[t,:, :(t * n_new)])
                recommended_items_2 = algo_2(training_data_2, noise[t,:, :(t * n_new)])

                # Merge the two algorithms' recommendation list
                recommended_items = recommended_items_1.copy()
                recommended_items[user_assignments] = recommended_items_2[user_assignments]

            #### Consumption step
            # Simulate user consumption
            # chosen_items reflects ID of the item each user chooses to consume, 
            # where -1 indicates that user does not consume any item
            if t <  params.initial_periods:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)
            else:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)

            # Update the user-item interaction in interaction_matrix and prev_consumed_items
            for user_id, chosen_item in enumerate(chosen_items):
                prev_consumed_items[user_id].append(chosen_item)
                if chosen_item != -1:
                    if user_assignments[user_id] == 0:
                            interaction_matrix_algo_1[user_id,chosen_item] = 1
                    else:
                            interaction_matrix_algo_2[user_id,chosen_item] = 1

        # Calculate the average take-up rate
        [avg_c_algo_1,avg_c_algo_2] = avg_take_up_rate_by_period(prev_consumed_items, user_assignments, params)
        avg_c_algo_1_list.append(np.mean(avg_c_algo_1[params.initial_periods:]))
        avg_c_algo_2_list.append(np.mean(avg_c_algo_2[params.initial_periods:]))
        if b < params.B:
            # Print the count of the simulation with replacement
            print(f"Simulation {b + 1} of {params.B} completed", end='\r', flush=True)
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
        else:
            print(f"Simulation {b + 1} of {params.B} completed")
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")

    # Calculate the average take up rate for algo_1 and algo_2 across simulations
    avg_algo_1 = np.zeros(4)
    avg_algo_1[0] = np.mean(avg_c_algo_1_list, axis=0)
    avg_algo_1[1] = np.quantile(avg_c_algo_1_list,0.025)
    avg_algo_1[2] = np.quantile(avg_c_algo_1_list,0.975)
    avg_algo_1[3] = np.var(avg_algo_1, axis=0)
    
    avg_algo_2 = np.zeros(4)
    avg_algo_2[0] = np.mean(avg_c_algo_2_list, axis=0)
    avg_algo_2[1] = np.quantile(avg_c_algo_2_list,0.025)
    avg_algo_2[2] = np.quantile(avg_c_algo_2_list,0.975)
    avg_algo_2[3] = np.var(avg_c_algo_2_list, axis=0)
    
    # Calculate the average Treatment effect (algo_1 - algo_2) across simulations
    avg_TE_list = np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list)
    avg_TE = np.zeros(4)
    avg_TE[0] = np.mean(avg_TE_list, axis=0)
    avg_TE[1] = np.quantile(avg_TE_list,0.025)
    avg_TE[2] = np.quantile(avg_TE_list,0.975)
    avg_TE[3] = np.var(avg_TE_list, axis=0)
    
    # Calculate the average percentage Treatment effect across simulations
    avg_pct_TE_list = (np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list))/np.array(avg_c_algo_1_list)
    avg_pct_TE = np.zeros(4)
    avg_pct_TE[0] = np.mean(avg_pct_TE_list, axis=0)
    avg_pct_TE[1] = np.quantile(avg_pct_TE_list,0.025)
    avg_pct_TE[2] = np.quantile(avg_pct_TE_list,0.975)
    avg_pct_TE[3] = np.var(avg_pct_TE_list, axis=0)


    return treatment_percentage, avg_algo_1, avg_algo_2, avg_TE, avg_pct_TE

#### Block 6: User-Corpus Co-diverted experiment

In [ ]:
def run_simulation_user_corpus_codiverted(params, user_item_utility, reserve_utilities,algo_1, algo_2, treatment_percentage):
    """
    User-corpus Co-diverted Method: 
    Both algo_1 and algo_2 learn from and act on the same user data.
    Algo_1 users and algo_2 users interact with their own pool of items, completely separated from each other.
    
    Args:
        params: Contains the enviroment set up.
        user_item_utility (function/array): A utility function or matrix providing user-utility values for items.
        reserve_utilities (array): Reserve_utilities for users.
        algo_1 (callable): The first recommendation algorithm; signature should match how it's called below.
        algo_2 (callable): The second recommendation algorithm; signature should match how it's called below.
        treatment_percentage (float): Percentage of treated user

    Returns:
        avg_c_algo_1 (np.array): [mean, 2.5% quantile, 97.5% quantile, variance] of take-up rates for algo_1.
        avg_c_algo_2 (np.array): Same structure as avg_c_algo_1 but for algo_2.
        avg_TE (np.array): Treatment effect (algo_1 - algo_2) summary stats.
        avg_pct_TE (np.array): Percentage-based treatment effect ((algo_1 - algo_2)/algo_1) summary stats.
    """

    # Initialize results lists
    avg_c_algo_1_list = []
    avg_c_algo_2_list = []

    # Pre-generate user assignment matrix for all simulation rounds (num of rounds = params.B)
    # Each row in user_assignments_matrix is a array of length num_users indicating which users are assigned to algo_2 (1) or algo_1 (0).
    user_assignments_matrix = np.random.choice(
            [0, 1], size=(params.B, params.num_users), p=[1-treatment_percentage, treatment_percentage]
        ).astype(bool)

    # Pre-generate item assignment matrix for all simulation rounds (num of rounds = params.B)
    # Each row in item_assignments_matrix is a array of length num_items indicating which users are assigned to algo_2 (1) or algo_1 (0).
    item_assignments_matrix = np.random.choice(
            [0, 1], size=(params.B, params.num_items), p=[1-treatment_percentage, treatment_percentage]
        ).astype(bool)


    # Number of new items in each period
    n_new = params.num_items_per_period

    # For each round of simulation
    for b in range(params.B):
        # Get user assignments for this simulation round
        user_assignments = user_assignments_matrix[b]
        # Get item assignments for this simulation round
        item_assignments = item_assignments_matrix[b]

        # Generate random noise for each period and user
        # Shape: (num_periods x num_users x (num_periods * num_items_per_period))
        noise = np.random.normal(0, 1,(params.num_periods,params.num_users, params.num_periods * params.num_items_per_period))
        
        # Initialize the user-item interaction matrix: 0 indicates no consumption, 1 indicates consumption
        # It’s essentially the interaction history that the algorithms use to learn user preferences.
        # Shape: (num_users x num_items)
        interaction_matrix = np.zeros((params.num_users, params.num_items))

        # Record previous consumption to keep track of all items consumed by each user after the initial period
        # Used later to calculate take up rate
        prev_consumed_items = [[] for _ in range(params.num_users)]

        for t in range(params.num_periods):

            #### Introduce new goods
            # list(range(t * n_new, (t + 1) * n_new)) generates a list of new item indices introduced at period t. 
            # For example, if n_new = 5 and t = 2, this would produce [10, 11, 12, 13, 14].
            # np.repeat(..., params.num_users, axis=0) replicates this list so that each user has the same set of new items to choose from initially.
            new_items = np.repeat([list(range(t * n_new, (t + 1) * n_new))], params.num_users, axis=0)
            
            # np.apply_along_axis(np.random.shuffle, 1, new_items) shuffles the order of these new item indices independently for each user.
            np.apply_along_axis(np.random.shuffle, 1, new_items)
            
            # Initialize recommended items list for each user
            recommended_items = [[] for _ in range(params.num_users)]

            #### Recommendation step (happens only after initial periods)
            # Update the training data every training_frequency periods:
            if t % params.training_frequency == 0 and t >= params.initial_periods:
                # Use item interation data for each user up to the current period (t * n_new) as training data
                # :(t * n_new) reflects all the items introduced up to the start of period t
                training_data = interaction_matrix[:, :(t * n_new)]
                
                # Recommended_items: Matrix of ranked item IDs recommended to each user.
                recommended_items_1 = algo_1(training_data, noise[t,:, :(t * n_new)])
                recommended_items_2 = algo_2(training_data, noise[t,:, :(t * n_new)])             
                
                # Merge the two algorithms' recommendation list
                recommended_items = recommended_items_1.copy()
                recommended_items[user_assignments] = recommended_items_2[user_assignments]

            #### Consumption step
            # Simulate user consumption
            # chosen_items reflects ID of the item each user chooses to consume, 
            # where -1 indicates that user does not consume any item
            if t <  params.initial_periods:
                chosen_items = consume_item_all_users_loop_user_corpus(recommended_items, new_items, user_item_utility, reserve_utilities, params, item_assignments, user_assignments)
            else:
                chosen_items = consume_item_all_users_loop_user_corpus(recommended_items, new_items, user_item_utility, reserve_utilities, params, item_assignments, user_assignments)

            # Update the user-item interaction in interaction_matrix and prev_consumed_items
            for user_id, chosen_item in enumerate(chosen_items):
                prev_consumed_items[user_id].append(chosen_item)
                if chosen_item != -1:
                    interaction_matrix[user_id,chosen_item] = 1
        # Calculate the average take-up rate
        [avg_c_algo_1,avg_c_algo_2] = avg_take_up_rate_by_period(prev_consumed_items, user_assignments, params)
        avg_c_algo_1_list.append(np.mean(avg_c_algo_1[params.initial_periods:]))
        avg_c_algo_2_list.append(np.mean(avg_c_algo_2[params.initial_periods:]))
        if b < params.B:
            # Print the count of the simulation with replacement
            print(f"Simulation {b + 1} of {params.B} completed", end='\r', flush=True)
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
        else:
            print(f"Simulation {b + 1} of {params.B} completed")
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")

    # Calculate the average take up rate for algo_1 and algo_2 across simulations
    avg_algo_1 = np.zeros(4)
    avg_algo_1[0] = np.mean(avg_c_algo_1_list, axis=0)
    avg_algo_1[1] = np.quantile(avg_c_algo_1_list,0.025)
    avg_algo_1[2] = np.quantile(avg_c_algo_1_list,0.975)
    avg_algo_1[3] = np.var(avg_algo_1, axis=0)
    
    avg_algo_2 = np.zeros(4)
    avg_algo_2[0] = np.mean(avg_c_algo_2_list, axis=0)
    avg_algo_2[1] = np.quantile(avg_c_algo_2_list,0.025)
    avg_algo_2[2] = np.quantile(avg_c_algo_2_list,0.975)
    avg_algo_2[3] = np.var(avg_c_algo_2_list, axis=0)
    
    # Calculate the average Treatment effect (algo_1 - algo_2) across simulations
    avg_TE_list = np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list)
    avg_TE = np.zeros(4)
    avg_TE[0] = np.mean(avg_TE_list, axis=0)
    avg_TE[1] = np.quantile(avg_TE_list,0.025)
    avg_TE[2] = np.quantile(avg_TE_list,0.975)
    avg_TE[3] = np.var(avg_TE_list, axis=0)
    
    # Calculate the average percentage Treatment effect across simulations
    avg_pct_TE_list = (np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list))/np.array(avg_c_algo_1_list)
    avg_pct_TE = np.zeros(4)
    avg_pct_TE[0] = np.mean(avg_pct_TE_list, axis=0)
    avg_pct_TE[1] = np.quantile(avg_pct_TE_list,0.025)
    avg_pct_TE[2] = np.quantile(avg_pct_TE_list,0.975)
    avg_pct_TE[3] = np.var(avg_pct_TE_list, axis=0)

    return treatment_percentage, avg_algo_1, avg_algo_2, avg_TE, avg_pct_TE

#### Block 7: Clustering experiment

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

def assign_clusters(preferences, max_clusters=10):
    # List to store average silhouette scores for different numbers of clusters
    silhouette_scores = []

    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=0).fit(preferences)
        silhouette_avg = silhouette_score(preferences, kmeans.labels_)
        silhouette_scores.append(silhouette_avg)

    # Find the number of clusters that gives the highest silhouette score

    optimal_clusters = np.argmax(silhouette_scores) + 2  # +2 because the range starts from 2
    # Re-run KMeans with the optimal number of clusters
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=0).fit(preferences)
    cluster_assignments = kmeans.labels_

    return cluster_assignments, optimal_clusters

def run_simulation_cluster(params, preferences,user_item_utility, reserve_utilities, algo_1, algo_2, treatment_percentage, cluster_shuffle_percent):
    """
    Cluster Method: 
    User data is divided into clusters based on predefined criteria.
    All users within a cluster are assigned the same recommendation algorithm.
    Both algo_1 and algo_2 learn from and act on the same user data.
    Users interact with the same pool of items.
    
    Args:
        params: Contains the enviroment set up.
        user_item_utility (function/array): A utility function or matrix providing user-utility values for items.
        reserve_utilities (array): Reserve_utilities for users.
        algo_1 (callable): The first recommendation algorithm; signature should match how it's called below.
        algo_2 (callable): The second recommendation algorithm; signature should match how it's called below.
        treatment_percentage (float): Percentage of treated user

    Returns:
        avg_c_algo_1 (np.array): [mean, 2.5% quantile, 97.5% quantile, variance] of take-up rates for algo_1.
        avg_c_algo_2 (np.array): Same structure as avg_c_algo_1 but for algo_2.
        avg_TE (np.array): Treatment effect (algo_1 - algo_2) summary stats.
        avg_pct_TE (np.array): Percentage-based treatment effect ((algo_1 - algo_2)/algo_1) summary stats.
    """
    # Initialize results lists
    avg_c_algo_1_list = []
    avg_c_algo_2_list = []

    # Pre-generate user assignment matrix
    # Same assignment matrix for all sub-simulation
    cluster_assignments, optimal_clusters = assign_clusters(preferences, max_clusters=10)
    
    ### Randomly reassigning cluster_shuffle_percent from each cluster
    # Initiate the new assignments after shuffling
    new_assignments = cluster_assignments.copy()
    
    if cluster_shuffle_percent > 0 and optimal_clusters > 1:
        for cluster_id in range(optimal_clusters):
            # Determine number of users to be shuffle for current cluster (n_shuffle_c)
            cluster_users = np.where(cluster_assignments == cluster_id)[0]
            n_cluster_users = len(cluster_users)
            n_shuffle_c = int(cluster_shuffle_percent * n_cluster_users)
            
            if n_shuffle_c > 0:
                # Randomly pick n_shuffle_c users to shuffle from this cluster
                shuffle_indices = np.random.choice(cluster_users, size=n_shuffle_c, replace=False)
                
                # Assign each of these users to a different cluster
                for idx in shuffle_indices:
                    old_cluster = cluster_assignments[idx]
                    # Exclude the old cluster from possible new clusters
                    possible_new_clusters = [c for c in range(optimal_clusters) if c != old_cluster]
                    # Update new user assignments
                    new_assignments[idx] = np.random.choice(possible_new_clusters)
    
    # Replace old assignments with new cluster assignments             
    cluster_assignments = new_assignments

    # Number of new items in each period
    n_new = params.num_items_per_period

    # For each round of simulation
    for b in range(params.B):

        # Generate random noise for each period and user
        # Shape: (num_periods x num_users x (num_periods * num_items_per_period))
        noise = np.random.normal(0, 1,(params.num_periods,params.num_users, params.num_periods * params.num_items_per_period))
        
        # Initialize the user-item interaction matrix: 0 indicates no consumption, 1 indicates consumption
        # It’s essentially the interaction history that the algorithms use to learn user preferences.
        # Shape: (num_users x num_items)
        interaction_matrix = np.zeros((params.num_users, params.num_items))

        # Record previous consumption to keep track of all items consumed by each user after the initial period
        # Used later to calculate take up rate
        prev_consumed_items = [[] for _ in range(params.num_users)]

        for t in range(params.num_periods):

            #### Introduce new goods
            # list(range(t * n_new, (t + 1) * n_new)) generates a list of new item indices introduced at period t. 
            # For example, if n_new = 5 and t = 2, this would produce [10, 11, 12, 13, 14].
            # np.repeat(..., params.num_users, axis=0) replicates this list so that each user has the same set of new items to choose from initially.
            new_items = np.repeat([list(range(t * n_new, (t + 1) * n_new))], params.num_users, axis=0)
            
            # np.apply_along_axis(np.random.shuffle, 1, new_items) shuffles the order of these new item indices independently for each user.
            np.apply_along_axis(np.random.shuffle, 1, new_items)
            
            # Initialize recommended items list for each user
            recommended_items = [[] for _ in range(params.num_users)]

            if t == params.initial_periods-1:

                # Randomly assign half the clusters to treatment based on the treatment percentage
                num_treatment_clusters = int(np.floor(optimal_clusters * treatment_percentage))
                treatment_clusters = np.random.choice(np.arange(optimal_clusters), size=num_treatment_clusters, replace=False)
                
                # Assign users to treatment based on their cluster membership
                # Since clusters vary in size, the actual percentage of users assigned to treatment may differ from the specified treatment_percentage.
                user_assignments = np.isin(cluster_assignments, treatment_clusters)

                # Calculate the actual percentage of users assigned to treatment
                actual_treatment_percentage = np.sum(user_assignments) / len(user_assignments)
                
            #### Recommendation step (happens only after initial periods)
            # Update the training data every training_frequency periods:
            if t % params.training_frequency == 0 and t >= params.initial_periods:
                # Use item interation data for each user up to the current period (t * n_new) as training data
                # :(t * n_new) reflects all the items introduced up to the start of period t
                training_data = interaction_matrix[:, :(t * n_new)]
                
                # Recommended_items: Matrix of ranked item IDs recommended to each user.
                recommended_items_1 = algo_1(training_data, noise[t,:, :(t * n_new)])
                recommended_items_2 = algo_2(training_data, noise[t,:, :(t * n_new)])

                # Merge the two algorithms' recommendation list
                recommended_items = recommended_items_1.copy()
                recommended_items[user_assignments] = recommended_items_2[user_assignments]

            #### Consumption step
            # Simulate user consumption
            # chosen_items reflects ID of the item each user chooses to consume, 
            # where -1 indicates that user does not consume any item
            if t <  params.initial_periods:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)
            else:
                chosen_items = consume_item_all_users_loop(recommended_items, new_items, user_item_utility, reserve_utilities, params)

            # Update the user-item interaction in interaction_matrix and prev_consumed_items
            for user_id, chosen_item in enumerate(chosen_items):
                prev_consumed_items[user_id].append(chosen_item)
                if chosen_item != -1:
                    interaction_matrix[user_id,chosen_item] = 1
            
        # Calculate the average take-up rate
        [avg_c_algo_1,avg_c_algo_2] = avg_take_up_rate_by_period(prev_consumed_items, user_assignments, params)
        avg_c_algo_1_list.append(np.mean(avg_c_algo_1[params.initial_periods:]))
        avg_c_algo_2_list.append(np.mean(avg_c_algo_2[params.initial_periods:]))
        if b < params.B:
            # Print the count of the simulation with replacement
            print(f"Simulation {b + 1} of {params.B} completed", end='\r', flush=True)
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
        else:
            print(f"Simulation {b + 1} of {params.B} completed")
            with open(params.output_file, 'a') as f:
                f.write(f"Simulation {b + 1} of {params.B} completed\n")
    
    # Calculate the average take up rate for algo_1 and algo_2 across simulations
    avg_algo_1 = np.zeros(4)
    avg_algo_1[0] = np.mean(avg_c_algo_1_list, axis=0)
    avg_algo_1[1] = np.quantile(avg_c_algo_1_list,0.025)
    avg_algo_1[2] = np.quantile(avg_c_algo_1_list,0.975)
    avg_algo_1[3] = np.var(avg_algo_1, axis=0)
    
    avg_algo_2 = np.zeros(4)
    avg_algo_2[0] = np.mean(avg_c_algo_2_list, axis=0)
    avg_algo_2[1] = np.quantile(avg_c_algo_2_list,0.025)
    avg_algo_2[2] = np.quantile(avg_c_algo_2_list,0.975)
    avg_algo_2[3] = np.var(avg_algo_2, axis=0)
    
    # Calculate the average Treatment effect (algo_1 - algo_2) across simulations
    avg_TE_list = np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list)
    avg_TE = np.zeros(4)
    avg_TE[0] = np.mean(avg_TE_list, axis=0)
    avg_TE[1] = np.quantile(avg_TE_list,0.025)
    avg_TE[2] = np.quantile(avg_TE_list,0.975)
    avg_TE[3] = np.var(avg_TE, axis=0)
    
    # Calculate the average percentage Treatment effect across simulations
    avg_pct_TE_list = (np.array(avg_c_algo_1_list)-np.array(avg_c_algo_2_list))/np.array(avg_c_algo_1_list)
    avg_pct_TE = np.zeros(4)
    avg_pct_TE[0] = np.mean(avg_pct_TE_list, axis=0)
    avg_pct_TE[1] = np.quantile(avg_pct_TE_list,0.025)
    avg_pct_TE[2] = np.quantile(avg_pct_TE_list,0.975)
    avg_pct_TE[3] = np.var(avg_pct_TE_list, axis=0)

    return actual_treatment_percentage, avg_algo_1, avg_algo_2, avg_TE, avg_pct_TE

#### Block 8: Simulation

In [ ]:
def run_experiment():
    ###########
    n_sim = 1000
    ###########
    params = Param(K=10, num_periods=100, num_users=100, num_items=1000,
                   sigma=1e-5, B=n_sim, random_seed=np.arange(30), per=50,
                   output_file="output.txt")
    num_items_per_period = int(params.num_items / params.num_periods)
    params.num_items_per_period = num_items_per_period
    params.training_frequency = 1
    params.initial_periods = 10
    params.gamma_pref = float(os.getenv('GAMMA_PREF', '1'))
    params.gamma_item = float(os.getenv('GAMMA_ITEM', '1'))
    params.pref_group = False
    treatment_percentage = float(os.getenv("TREATMENT_PERCENT", "0.5"))
    cluster_shuffle_percentage = float(os.getenv("CLUSTER_SHUFFLE_PERCENTAGE", "0.0"))

    seed = 13034
    np.random.seed(seed)

    cluster_size = int(os.getenv("CLUSTER_SIZE", "10"))

    # Generate primitives
    preferences = generate_user_preferences_cluster_with_size(params, cluster_size)
    characteristics = generate_item_char_cluster(params)
    user_item_utility = generate_values(characteristics, preferences, params)
    reserve_utility = calculate_reserve_utilities(user_item_utility, params)

    # Define algorithms using a dictionary
    algorithms = {
        "Item": Item_based_CF,
        "User": User_based_CF,
        "Random": Random_alg,
        "Ideal": lambda training_data, noise: Ideal_alg(
            training_data, user_item_utility, noise
        ),
    }

    # Define method functions using a dictionary
    method_functions = {
        "Naive": run_simulation_naive,
        "Data-diverted": run_simulation_data_diverted, 
        "Cluster": run_simulation_cluster,
        "User-corpus": run_simulation_user_corpus_codiverted
    }

    # Methods and combinations
    methods = ["Naive", "Data-diverted", "Cluster", "User-corpus"]
    combinations = [
        ("Item", "Ideal"),
        ("Item", "Random"),
        ("Item", "User"),
        ("Ideal", "Item"),
        ("Ideal", "Random"),
        ("Ideal", "User"),
        ("Random", "Item"),
        ("Random", "Ideal"),
        ("Random", "User"),
        ("User", "Ideal"),
        ("User", "Random"),
        ("User", "Item"),
    ]
    algo_list = ["Item", "User", "Random", "Ideal"]

    # Results storage
    results = []
    file_path = f"./results/Simulation_result.csv"

    # Reference simulations
    for algo_name in algo_list:
        alg = algorithms.get(algo_name)
        TUR, TUR_2, TE, pct_TE = run_simulation_ref(
            params, user_item_utility, reserve_utility, alg, alg
        )
        add_result(
            results,
            algo_name,
            algo_name,
            "Ref",
            params.gamma_pref,
            0,
            cluster_shuffle_percentage,
            TUR,
            TUR_2,
            TE,
            pct_TE
        )

    # Other methods
    for method in methods:
        method_function = method_functions.get(method)
        for algo1_name, algo2_name in combinations:
            algo1 = algorithms.get(algo1_name)
            algo2 = algorithms.get(algo2_name)

            # Determine if 'preferences' is needed
            if method == "Cluster":
                TP, TUR1, TUR2, TE, pct_TE = method_function(
                    params,
                    preferences,
                    user_item_utility,
                    reserve_utility,
                    algo1,
                    algo2,
                    treatment_percentage,
                    cluster_shuffle_percentage,
                )
            else:
                TP, TUR1, TUR2, TE, pct_TE = method_function(
                    params,
                    user_item_utility,
                    reserve_utility,
                    algo1,
                    algo2,
                    treatment_percentage,
                )

            add_result(
                results,
                algo1_name,
                algo2_name,
                method,
                params.gamma_pref,
                TP,
                cluster_shuffle_percentage,
                TUR1,
                TUR2,
                TE,
                pct_TE,
            )

    # Save or append results to CSV
    add_and_save_results(results, file_path)
    results_df = pd.DataFrame(results)

def add_and_save_results(results, file_path):
    # Check if the CSV file already exists
    file_exists = os.path.exists(file_path)

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Append to the CSV file without writing the header if the file already exists
    results_df.to_csv(file_path, mode='a', header=not file_exists, index=False)

def add_result(results, algo1, algo2, method, gamma_pref, treatment_percentage, cluster_shuffle_percentage, TUR_algo_1, TUR_algo_2, TE, pct_TE):
    results.append({
        "algo1": algo1,
        "algo2": algo2,
        "method": method,
        "gamma_pref": gamma_pref,
        "treatment_percentage": treatment_percentage,
        "cluster_shuffle_percentage": cluster_shuffle_percentage,
        "TUR_algo_1": TUR_algo_1,
        "TUR_algo_2": TUR_algo_2,
        "TE": TE,
        "Percentage_TE": pct_TE
    })

In [ ]:
run_experiment()